In [ ]:
import freud
import gsd.hoomd
import numpy as np
import h5py

import matplotlib.pyplot as plt
from matplotlib.cm import viridis

In [ ]:
#Useful functions for letter
def moving_average(x, average_window=10):
    num_points = len(x)-average_window
    average = []
    for i in range(num_points):
        average.append(np.mean(x[i:i+average_window]))
    return average


def get_derivative(x,y):
    num_points = len(x)
    derivative = []
    for i in range(1,num_points):
        diff = (y[i]-y[i-1])/(x[i]-x[i-1])
        derivative.append(diff)
        
    return derivative

def basic_inline_visual(filename, frame):
    with gsd.hoomd.open(filename) as traj:
        frame = traj[frame]

        figure = plt.figure(figsize=(10,10))
        ax = figure.add_subplot(projection='3d')
        pos = frame.particles.position
        
        ax.scatter(pos[:,0],pos[:,1], pos[:,2])

In this notebook, we will examine how the RDF changes as a system melts and then use potential energy to identify a phase change.

# The RDF through phase transitions

We will start by using the basic RDF code from Simple Fluids 1 to visualize what happens as the system melts.

The code below will visualize one pressure with stacked RDFs. What happens as you change pressure?

In [ ]:
n_bins = 100
r_max = 4.0
all_rdfs = []

with gsd.hoomd.open('trajectory_1.0.gsd','r') as traj:
    #To simplify, we will only look at every 10 frames
    #Remember that the first 10 frames are pressure equilibration
    for frame in traj[0:-1:10]:
        box = frame.configuration.box
        positions = frame.particles.position
        rdf = freud.density.RDF(bins = n_bins, r_max = r_max)
        rdf.compute((box, positions))
        all_rdfs.append(rdf.rdf)

x_data = rdf.bin_centers
for i in range(len(all_rdfs)):
    plt.plot(x_data, all_rdfs[i]+i,color = viridis(i/10) )
    
plt.xlabel('Distance')
plt.ylabel('g(r) (A. U.)')
plt.title('Change in RDF with melting')

You can identify phase transitions through the RDF. How would you do that here?

# Identifying Phase Transitions

Identifying phase changes well is extremely important, as it is highly effected by aspects like heating rate and nucleation. Here, we will identify phase changes through *melting* simulations because a melting transition typically has a much lower energy barrier than crystallization.

The easiest way to identify a phase change is through a discontinuity in some thermodynamic quantity. A first-order phase transition like melting will exhibit a large jump in potential energy, heat capacity, volume, and many other properties right at the melting point.

Here we will use potential energy to measure the transition.

First, we need to load the h5 files that we saved using the h5py utility. We can access specific quantities like this:

In [ ]:
hdf5_file = h5py.File(name='log_1.0.h5', mode='r')

energies = hdf5_file['hoomd-data/md/compute/ThermodynamicQuantities/potential_energy'][:]
temperatures = hdf5_file['hoomd-data/md/compute/ThermodynamicQuantities/kinetic_temperature'][:]
time = hdf5_file['hoomd-data/Simulation/timestep'][:]

In [ ]:
plt.figure(figsize = (8,5))
plt.scatter(temperatures,energies)
plt.xlabel('Temperature (kT)');
plt.ylabel('Energy (E)');
plt.title('Visualizing a Phase Transition')

At what temperature do you estimate the phase transition is taking place?

To determine exactly what temperature this is, we can use a derivative. The maximum of the derivative will be, roughly, the melting point. We will take the derivative with time instead of temperature, because the temperature fluctuates near the melting point and introduces a lot of noise.

Note that this is an *extremely* rough measure of the melting temperature. There are other methods to find the melting point, like fitting a function. Or we could fit a peak to the derivative instead of simply taking the maximum. Implementing one of these more correct methods will be an excersise for the end of this session.

In the example below, note the extremely high maximum. That's the signature we're watching for.

In [ ]:
dE = get_derivative(time,energies)
dE_maximum = np.argmax(dE)
rough_transition_temperature = np.round(temperatures[dE_maximum],2)

plt.scatter(time[1:], dE)
plt.ylabel('Energy/timestep');
plt.xlabel('Timestep');
plt.ylabel('Derivative of PE');

plt.title('Melting temperature: {}'.format(rough_transition_temperature))

When identifying a phase transition programmatically, it's always a good idea to use a sanity check. For example, plot the RDF directly before and after the phase transition and check that it behaves as you think it should for melting. For instance, it could be that there are multiple phase transitions in one simulation and you have captured a different one.

In [ ]:
with gsd.hoomd.open('trajectory_1.0.gsd','r') as traj:
    liquid_frame = ?
    solid_frame = ?

rdf = freud.density.RDF(bins = n_bins, r_max = r_max)

rdf.compute((liquid_frame.configuration.box, liquid_frame.particles.position))
rdf.plot()


rdf.compute((solid_frame.configuration.box, solid_frame.particles.position))
rdf.plot()

# Generate a Phase Diagram

Now use the simulations you ran to map out a rough coexistence curve for the (Pressure, Temperature) plane.

In [ ]:
pressures = np.arange(1.0,6.0)

In [ ]:
melting_points = []

for pressure in pressures:
    hdf5_file = h5py.File(name='log_{}.h5'.format(pressure), mode='r')
    energies = hdf5_file['hoomd-data/md/compute/ThermodynamicQuantities/potential_energy'][:]
    temperatures = hdf5_file['hoomd-data/md/compute/ThermodynamicQuantities/kinetic_temperature'][:]
    time = hdf5_file['hoomd-data/Simulation/timestep'][:]
    
    #Get the derivative
    #Find the maximum and record it

In [ ]:
plt.figure(figsize = (5,5))

plt.scatter(pressures, melting_points)

plt.xlabel('Temperature (kT)')
plt.ylabel('Pressure (E/$D^3$)')
plt.title('P-T Phase Diagram')
plt.ylim(0.5,2.0)

Compare your results to those of your neighbor. What does this tell you about statistical randomness?

Now that we have determined a very rough phase diagram, how can you improve the calculation? Brainstorm at least one method (that doesn't involve running more simulations) and implement it.

*Hint: the moving_average function implemented at the top of this notebook may be useful. Implemented correctly, it will smooth out fluctuations in the recorded temperature and energy.*